In [2]:
import math
import time

# Classes para métricas de esforço computacional
class Metricas:
    def __init__(self):
        self.f_evals = 0  # Avaliações de função
        self.ops = 0      # Operações aritméticas
        self.logic = 0    # Decisões lógicas
        self.iteracoes = 0
        self.tempo_total = 0

def medir_tempo(func):
    def wrapper(*args, **kwargs):
        inicio = time.perf_counter()
        resultado = func(*args, **kwargs)
        fim = time.perf_counter()
        if len(args) > 0 and isinstance(args, Metricas):
             args.tempo_total += (fim - inicio)
        return resultado
    return wrapper

In [3]:
def bisseccao(f, a, b, eps, metricas):
    # ALGORITMO 1 [7, 14]
    
    # 1) Dados iniciais: a) intervalo [a, b], b) precisão eps
    # (Passados como argumentos)
    
    # 2) Se (b - a) < eps, então escolha para x_bar qualquer x em [a, b]. FIM.
    metricas.ops += 1; metricas.logic += 1
    if (b - a) < eps:
        return (a + b) / 2
    
    # 3) k = 1
    k = 1
    metricas.iteracoes = k
    
    while True:
        # 4) M = f(a)
        M = f(a)
        metricas.f_evals += 1
        
        # 5) x = (a + b) / 2
        x = (a + b) / 2
        metricas.ops += 2
        
        # 6) Se Mf(x) > 0, faça a = x. Vá para o passo 8.
        fx = f(x)
        metricas.f_evals += 1
        metricas.ops += 1; metricas.logic += 1
        
        if M * fx > 0:
            a = x
        else:
            # 7) b = x
            b = x
            
        # 8) Se (b - a) < eps, escolha para x_bar qualquer x em [a, b]. FIM.
        metricas.ops += 1; metricas.logic += 1
        if (b - a) < eps:
            metricas.iteracoes = k
            return x # Retornamos x como aproximação final
            
        # 9) k = k + 1. Volte para o passo 5.
        k = k + 1
        metricas.iteracoes = k

In [4]:
def posicao_falsa(f, a, b, eps1, eps2, metricas):
    # ALGORITMO 2 [8, 9]
    
    # 1) Dados iniciais: a) intervalo [a, b], b) precisões eps1 e eps2
    
    # 2) Se (b - a) < eps1, escolha x em [a, b]. FIM.
    metricas.ops += 1; metricas.logic += 1
    if (b - a) < eps1:
        return (a + b) / 2
    
    # ou se |f(a)| < eps2 ou |f(b)| < eps2, escolha a ou b. FIM.
    fa = f(a)
    fb = f(b)
    metricas.f_evals += 2
    metricas.logic += 2
    if abs(fa) < eps2: return a
    if abs(fb) < eps2: return b
    
    # 3) k = 1
    k = 1
    
    while True:
        # 4) M = f(a)
        M = f(a)
        metricas.f_evals += 1
        
        # 5) x = (af(b) - bf(a)) / (f(b) - f(a))
        fb = f(b) # Recalculando ou reutilizando (o algoritmo sugere usar os valores disponíveis)
        fa = M
        metricas.f_evals += 1 # Contando f(b)
        x = (a * fb - b * fa) / (fb - fa)
        metricas.ops += 5
        
        # 6) Se |f(x)| < eps2, escolha x_bar = x. FIM.
        fx = f(x)
        metricas.f_evals += 1
        metricas.logic += 1
        if abs(fx) < eps2:
            metricas.iteracoes = k
            return x
            
        # 7) Se Mf(x) > 0, faça a = x. Vá para o passo 9.
        metricas.ops += 1; metricas.logic += 1
        if M * fx > 0:
            a = x
        else:
            # 8) b = x
            b = x
            
        # 9) Se b - a < eps1, escolha x_bar em (a, b). FIM.
        metricas.ops += 1; metricas.logic += 1
        if abs(b - a) < eps1: # abs usado pois a pode ser > b dependendo da implementação, mas aqui segue padrão
             metricas.iteracoes = k
             return x
             
        # 10) k = k + 1. Volte ao passo 5.
        k = k + 1
        metricas.iteracoes = k

In [5]:
def ponto_fixo(f_original, phi, x0, eps1, eps2, metricas):
    # ALGORITMO 3 [10]
    
    # 1) Dados iniciais: x0, eps1, eps2
    
    # 2) Se |f(x0)| < eps1, faça x_bar = x0. FIM.
    fx0 = f_original(x0)
    metricas.f_evals += 1
    metricas.logic += 1
    if abs(fx0) < eps1:
        return x0
        
    # 3) k = 1
    k = 1
    
    while True:
        # 4) x1 = phi(x0)
        x1 = phi(x0)
        metricas.f_evals += 1 # Avaliação de phi contada como função
        
        # 5) Se |f(x1)| < eps1 ou |x1 - x0| < eps2, faça x_bar = x1. FIM.
        fx1 = f_original(x1)
        metricas.f_evals += 1
        metricas.ops += 1; metricas.logic += 2
        
        if abs(fx1) < eps1 or abs(x1 - x0) < eps2:
            metricas.iteracoes = k
            return x1
            
        # 6) x0 = x1
        x0 = x1
        
        # 7) k = k + 1. Volte ao passo 4.
        k = k + 1
        metricas.iteracoes = k
        if k > 100: return x1 # Proteção contra loop infinito (divergência)

In [6]:
def newton_raphson(f, df, x0, eps1, eps2, metricas):
    # ALGORITMO 4 [11, 12]
    
    # 1) Dados iniciais: x0, eps1, eps2
    
    # 2) Se |f(x0)| < eps1, faça x_bar = x0. FIM.
    fx0 = f(x0)
    metricas.f_evals += 1
    metricas.logic += 1
    if abs(fx0) < eps1:
        return x0
        
    # 3) k = 1
    k = 1
    
    while True:
        # 4) x1 = x0 - f(x0)/f'(x0)
        # Nota: f(x0) já foi calculado ou atualizado, mas f'(x0) precisa calcular
        dfx0 = df(x0)
        metricas.f_evals += 1 # Derivada conta como avaliação
        metricas.ops += 2 # Divisão e subtração
        x1 = x0 - fx0 / dfx0
        
        # 5) Se |f(x1)| < eps1 ou |x1 - x0| < eps2, faça x_bar = x1. FIM.
        fx1 = f(x1)
        metricas.f_evals += 1
        metricas.ops += 1; metricas.logic += 2
        
        if abs(fx1) < eps1 or abs(x1 - x0) < eps2:
            metricas.iteracoes = k
            return x1
            
        # 6) x0 = x1
        x0 = x1
        fx0 = fx1 # Atualiza f(x0) para a próxima iteração sem recalcular (otimização implícita)
        
        # 7) k = k + 1. Volte ao passo 4.
        k = k + 1
        metricas.iteracoes = k
        if k > 100: return x1

In [7]:
def secante(f, x0, x1, eps1, eps2, metricas):
    # ALGORITMO 5 [6, 13]
    
    # 1) Dados iniciais: x0, x1, eps1, eps2
    
    # 2) Se |f(x0)| < eps1, faça x_bar = x0. FIM.
    fx0 = f(x0)
    metricas.f_evals += 1
    metricas.logic += 1
    if abs(fx0) < eps1: return x0
    
    # 3) Se |f(x1)| < eps1 ou |x1 - x0| < eps2, faça x_bar = x1. FIM.
    fx1 = f(x1)
    metricas.f_evals += 1
    metricas.ops += 1; metricas.logic += 2
    if abs(fx1) < eps1 or abs(x1 - x0) < eps2: return x1
    
    # 4) k = 1
    k = 1
    
    while True:
        # 5) x2 = x1 - (f(x1) / (f(x1) - f(x0))) * (x1 - x0)
        # Otimizando para a fórmula do livro: x_k - [f(xk) / (f(xk)-f(xk-1))] * (xk - xk-1)
        # Operações: 2 sub, 1 div, 1 mult, 1 sub final = 5 ops
        metricas.ops += 5
        x2 = x1 - (fx1 / (fx1 - fx0)) * (x1 - x0)
        
        # 6) Se |f(x2)| < eps1 ou |x2 - x1| < eps2, faça x_bar = x2. FIM.
        fx2 = f(x2)
        metricas.f_evals += 1
        metricas.ops += 1; metricas.logic += 2
        
        if abs(fx2) < eps1 or abs(x2 - x1) < eps2:
            metricas.iteracoes = k
            return x2
            
        # 7) x0 = x1, x1 = x2
        x0 = x1
        fx0 = fx1
        x1 = x2
        fx1 = fx2
        
        # 8) k = k + 1. Volte ao passo 5.
        k = k + 1
        metricas.iteracoes = k
        if k > 100: return x2


In [16]:
import pandas as pd
import math

# Exemplo 18
# f18(x) = e^(-x)^2 - math.cos(x)
# xi = (1,2)
# eps1 = eps2 = 10e-4
# Dados iniciais (Bissecção: a=1, b=2; Posição Falsa: a=1, b=2; MPF phi(x) = math.cos(x)-math.exp(-x**2)+x)
# xBarra
# f(xBarra)